# Regression model with clusters-age

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from datetime import timedelta
import re
from dateutil.parser import parse
import string
import warnings
warnings.filterwarnings('ignore')
import plotly.graph_objects as go
from sklearn.metrics import mean_absolute_percentage_error
import missingno as msno
from plotly.subplots import make_subplots
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
import scipy
import researchpy as rp
from matplotlib.offsetbox import AnchoredText
#import pacmap
from sklearn.compose import ColumnTransformer
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
import plotly.express as px
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split 
# Import Halving Grid Search
from sklearn.experimental import enable_halving_search_cv 
from sklearn.model_selection import HalvingGridSearchCV
import xgboost as xgb
from scipy.stats import kurtosis,skew
from numpy import mean,sqrt,square
from scipy.fftpack import fft, fftfreq
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import PredefinedSplit
import pickle
sns.set_style("darkgrid")
pd.set_option('display.max_columns', None)
pd.options.display.max_colwidth = 100

### read data

In [2]:
# read df full csv
df_features0=pd.read_csv('../../../data/processed data/df_feature_engineering_18December2022.csv', parse_dates=['Timestamp'], index_col='Timestamp')#["2020":"2021"]
df_features0["TPH"]=df_features0["TPH"].shift(-10)
df_features0["loss of TPH"]=df_features0["loss of TPH"].shift(-10)
df_features0.dropna(inplace=True)
df_features0.sort_index(inplace=True)
df_features0.shape

(859325, 403)

In [3]:
# Verificando si hay duplicados
u=[i for i in df_features0.columns if (not re.match(".*water",i)) and (not re.match(".*power",i)) and (not re.match(".*delta HH TPH",i)) ]
y=[i for i in df_features0.columns if  re.match(".*sag power index",i)] 
u=y+u
df_features=df_features0[u].copy()
df_features.head(1)

,sag power index,mean_sag power index_10,max_sag power index_10,min_sag power index_10,rms_sag power index_10,var_sag power index_10,mean_sag power index_5,max_sag power index_5,min_sag power index_5,rms_sag power index_5,var_sag power index_5,mean_sag power index_3,max_sag power index_3,min_sag power index_3,rms_sag power index_3,var_sag power index_3,sag power index_(t-1),sag power index_(t-2),sag power index_(t-3),sag power index_(t-4),sag power index_(t-5),pyrite law,chalcopyrite law,chalcocite law,covelin law,crusher index,ball work index,bornite law,charge cell,TPH,speed,HH TPH,granulometry,Edad,loss of TPH,solid percentage,HH charge cell,LL charge cell,delta HH charge cell,delta LL charge cell,mean_pyrite law_10,max_pyrite law_10,min_pyrite law_10,rms_pyrite law_10,var_pyrite law_10,mean_pyrite law_5,max_pyrite law_5,min_pyrite law_5,rms_pyrite law_5,var_pyrite law_5,mean_pyrite law_3,max_pyrite law_3,min_pyrite law_3,rms_pyrite law_3,var_pyrite law_3,mean_chalcopyrite law_10,max_chalcopyrite law_10,min_chalcopyrite law_10,rms_chalcopyrite law_10,var_chalcopyrite law_10,mean_chalcopyrite law_5,max_chalcopyrite law_5,min_chalcopyrite law_5,rms_chalcopyrite law_5,var_chalcopyrite law_5,mean_chalcopyrite law_3,max_chalcopyrite law_3,min_chalcopyrite law_3,rms_chalcopyrite law_3,var_chalcopyrite law_3,mean_chalcocite law_10,max_chalcocite law_10,min_chalcocite law_10,rms_chalcocite law_10,var_chalcocite law_10,mean_chalcocite law_5,max_chalcocite law_5,min_chalcocite law_5,rms_chalcocite law_5,var_chalcocite law_5,mean_chalcocite law_3,max_chalcocite law_3,min_chalcocite law_3,rms_chalcocite law_3,var_chalcocite law_3,mean_covelin law_10,max_covelin law_10,min_covelin law_10,rms_covelin law_10,var_covelin law_10,mean_covelin law_5,max_covelin law_5,min_covelin law_5,rms_covelin law_5,var_covelin law_5,mean_covelin law_3,max_covelin law_3,min_covelin law_3,rms_covelin law_3,var_covelin law_3,mean_crusher index_10,max_crusher index_10,min_crusher index_10,rms_crusher index_10,var_crusher index_10,mean_crusher index_5,max_crusher index_5,min_crusher index_5,rms_crusher index_5,var_crusher index_5,mean_crusher index_3,max_crusher index_3,min_crusher index_3,rms_crusher index_3,var_crusher index_3,mean_ball work index_10,max_ball work index_10,min_ball work index_10,rms_ball work index_10,var_ball work index_10,mean_ball work index_5,max_ball work index_5,min_ball work index_5,rms_ball work index_5,var_ball work index_5,mean_ball work index_3,max_ball work index_3,min_ball work index_3,rms_ball work index_3,var_ball work index_3,mean_bornite law_10,max_bornite law_10,min_bornite law_10,rms_bornite law_10,var_bornite law_10,mean_bornite law_5,max_bornite law_5,min_bornite law_5,rms_bornite law_5,var_bornite law_5,mean_bornite law_3,max_bornite law_3,min_bornite law_3,rms_bornite law_3,var_bornite law_3,mean_charge cell_10,max_charge cell_10,min_charge cell_10,rms_charge cell_10,var_charge cell_10,mean_charge cell_5,max_charge cell_5,min_charge cell_5,rms_charge cell_5,var_charge cell_5,mean_charge cell_3,max_charge cell_3,min_charge cell_3,rms_charge cell_3,var_charge cell_3,mean_speed_10,max_speed_10,min_speed_10,rms_speed_10,var_speed_10,mean_speed_5,max_speed_5,min_speed_5,rms_speed_5,var_speed_5,mean_speed_3,max_speed_3,min_speed_3,rms_speed_3,var_speed_3,mean_granulometry_10,max_granulometry_10,min_granulometry_10,rms_granulometry_10,var_granulometry_10,mean_granulometry_5,max_granulometry_5,min_granulometry_5,rms_granulometry_5,var_granulometry_5,mean_granulometry_3,max_granulometry_3,min_granulometry_3,rms_granulometry_3,var_granulometry_3,mean_solid percentage_10,max_solid percentage_10,min_solid percentage_10,rms_solid percentage_10,var_solid percentage_10,mean_solid percentage_5,max_solid percentage_5,min_solid percentage_5,rms_solid percentage_5,var_solid percentage_5,mean_solid percentage_3,max_solid percentage_3,min_solid percentage_3,rms_solid percentage_3,var_solid percentage_3,mean_HH charge cell_10,max_HH charge 

### read kmeans

In [4]:
# Se carga clasificador de mineralogia
#kmeans = pickle.load(open('..//..//..//models//pickles//Modelo update Accenture//Clasificador_dispatch_n_4.pkl', 'rb'))

In [5]:
# Se carga clasificador de mineralogia
kmeans = pickle.load(open('..//..//..//models//pickles//Clasificador_dispatch_n_4.pkl', 'rb'))

In [6]:
# Predict cluster
df_features["cluster"]=kmeans.predict(df_features[['crusher index',
                                              'sag power index',
                                               'ball work index',
                                              'chalcopyrite law',
                                              'pyrite law',
                                              'chalcocite law',
                                              'bornite law',
                                              'covelin law']])

### Caracterización de clusters

In [7]:
df_features["cluster"].value_counts()

0    392990
2    298081
3    152649
1     15605
Name: cluster, dtype: int64

In [8]:
df_features[['crusher index',
                'sag power index',
                'ball work index',
                'chalcopyrite law',
                'pyrite law',
                'chalcocite law',
                'bornite law',
                'covelin law',"cluster"]].groupby("cluster").mean()

,crusher index,sag power index,ball work index,chalcopyrite law,pyrite law,chalcocite law,bornite law,covelin law
cluster,,,,,,,,
0,20.816319,111.607845,16.592977,1.242311,2.657498,0.286693,0.011805,0.020925
1,18.640304,156.078058,17.299324,1.455064,2.817043,0.127139,0.010550,0.004418
2,24.466008,91.524171,16.018552,1.079985,2.582236,0.431586,0.018511,0.033891
3,19.964221,129.870814,16.765366,1.102959,2.127651,0.255450,0.007315,0.015369


* cluster 0: blando-duro  
* cluster 1: muy duro
* cluster 2: blando
* cluster 3: duro

### Categoria cluster-edad

In [9]:
df_features["Edad"].value_counts()

2.0    383913
0.0    243461
1.0    231951
Name: Edad, dtype: int64

In [10]:
df_features["cluster-edad"]=df_features.apply(lambda x: str(int(x["cluster"]))+str(int(x["Edad"])),axis=1)

In [11]:
df_features["cluster-edad"].value_counts()

02    183252
22    131329
01    107931
00    101807
20     93329
21     73423
32     63960
30     45815
31     42874
11      7723
12      5372
10      2510
Name: cluster-edad, dtype: int64

### Definición de variables de modelo

In [12]:
variables_interes=['charge cell', 'TPH', 'speed','HH TPH', 'granulometry','loss of TPH','solid percentage',
"HH charge cell","LL charge cell","delta HH charge cell","delta LL charge cell"]

In [81]:
features_model=[i for i in df_features.columns if  re.match(".*charge cell",i) or re.match(".*TPH",i) or  re.match(".*speed",i) or  re.match(".*granulometry",i) or  re.match(".*solid percentage",i)]+["cluster-edad","cluster","Edad"]

In [82]:
df_features_final=df_features[features_model].copy()

### Train-test split

In [83]:
# Entrenamiento
entrenamiento=pd.concat([df_features_final["2020-01":"2020-03"],df_features_final["2020-05"],df_features_final["2020-07":"2020-12"],df_features_final["2021-01":"2021-08"],df_features_final["2021-10"],df_features_final["2021-12"]])
entrenamiento.head(1)

,charge cell,TPH,speed,HH TPH,granulometry,loss of TPH,solid percentage,HH charge cell,LL charge cell,delta HH charge cell,delta LL charge cell,mean_charge cell_10,max_charge cell_10,min_charge cell_10,rms_charge cell_10,var_charge cell_10,mean_charge cell_5,max_charge cell_5,min_charge cell_5,rms_charge cell_5,var_charge cell_5,mean_charge cell_3,max_charge cell_3,min_charge cell_3,rms_charge cell_3,var_charge cell_3,mean_speed_10,max_speed_10,min_speed_10,rms_speed_10,var_speed_10,mean_speed_5,max_speed_5,min_speed_5,rms_speed_5,var_speed_5,mean_speed_3,max_speed_3,min_speed_3,rms_speed_3,var_speed_3,mean_granulometry_10,max_granulometry_10,min_granulometry_10,rms_granulometry_10,var_granulometry_10,mean_granulometry_5,max_granulometry_5,min_granulometry_5,rms_granulometry_5,var_granulometry_5,mean_granulometry_3,max_granulometry_3,min_granulometry_3,rms_granulometry_3,var_granulometry_3,mean_solid percentage_10,max_solid percentage_10,min_solid percentage_10,rms_solid percentage_10,var_solid percentage_10,mean_solid percentage_5,max_solid percentage_5,min_solid percentage_5,rms_solid percentage_5,var_solid percentage_5,mean_solid percentage_3,max_solid percentage_3,min_solid percentage_3,rms_solid percentage_3,var_solid percentage_3,mean_HH charge cell_10,max_HH charge cell_10,min_HH charge cell_10,rms_HH charge cell_10,var_HH charge cell_10,mean_HH charge cell_5,max_HH charge cell_5,min_HH charge cell_5,rms_HH charge cell_5,var_HH charge cell_5,mean_HH charge cell_3,max_HH charge cell_3,min_HH charge cell_3,rms_HH charge cell_3,var_HH charge cell_3,mean_LL charge cell_10,max_LL charge cell_10,min_LL charge cell_10,rms_LL charge cell_10,var_LL charge cell_10,mean_LL charge cell_5,max_LL charge cell_5,min_LL charge cell_5,rms_LL charge cell_5,var_LL charge cell_5,mean_LL charge cell_3,max_LL charge cell_3,min_LL charge cell_3,rms_LL charge cell_3,var_LL charge cell_3,mean_delta HH charge cell_10,max_delta HH charge cell_10,min_delta HH charge cell_10,rms_delta HH charge cell_10,var_delta HH charge cell_10,mean_delta HH charge cell_5,max_delta HH charge cell_5,min_delta HH charge cell_5,rms_delta HH charge cell_5,var_delta HH charge cell_5,mean_delta HH charge cell_3,max_delta HH charge cell_3,min_delta HH charge cell_3,rms_delta HH charge cell_3,var_delta HH charge cell_3,mean_delta LL charge cell_10,max_delta LL charge cell_10,min_delta LL charge cell_10,rms_delta LL charge cell_10,var_delta LL charge cell_10,mean_delta LL charge cell_5,max_delta LL charge cell_5,min_delta LL charge cell_5,rms_delta LL charge cell_5,var_delta LL charge cell_5,mean_delta LL charge cell_3,max_delta LL charge cell_3,min_delta LL charge cell_3,rms_delta LL charge cell_3,var_delta LL charge cell_3,charge cell_(t-1),charge cell_(t-2),charge cell_(t-3),charge cell_(t-4),charge cell_(t-5),speed_(t-1),speed_(t-2),speed_(t-3),speed_(t-4),speed_(t-5),granulometry_(t-1),granulometry_(t-2),granulometry_(t-3),granulometry_(t-4),granulometry_(t-5),solid percentage_(t-1),solid percentage_(t-2),solid percentage_(t-3),solid percentage_(t-4),solid percentage_(t-5),HH charge cell_(t-1),HH charge cell_(t-2),HH charge cell_(t-3),HH charge cell_(t-4),HH charge cell_(t-5),LL charge cell_(t-1),LL charge cell_(t-2),LL charge cell_(t-3),LL charge cell_(t-4),LL charge cell_(t-5),delta HH charge cell_(t-1),delta HH charge cell_(t-2),delta HH charge cell_(t-3),delta HH charge cell_(t-4),delta HH charge cell_(t-5),delta LL charge cell_(t-1),delta LL charge cell_(t-2),delta LL charge cell_(t-3),delta LL charge cell_(t-4),delta LL charge cell_(t-5),cluster-edad,cluster,Edad
Timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-01-10 01:09:00,820.483684,4102.411862,9.312051,4100.0,42.044058,0.0,71.462459,850.0,770.0,29.516316,50.483684,823.020917,824.839407,820.483684,823.022047,1.860281,822.060734,823.654031,820.483684,822.061503,1.265025,8

In [84]:
# Validación
validacion=pd.concat([df_features_final["2020-06"],df_features_final["2021-11"]])
validacion.head(1)

,charge cell,TPH,speed,HH TPH,granulometry,loss of TPH,solid percentage,HH charge cell,LL charge cell,delta HH charge cell,delta LL charge cell,mean_charge cell_10,max_charge cell_10,min_charge cell_10,rms_charge cell_10,var_charge cell_10,mean_charge cell_5,max_charge cell_5,min_charge cell_5,rms_charge cell_5,var_charge cell_5,mean_charge cell_3,max_charge cell_3,min_charge cell_3,rms_charge cell_3,var_charge cell_3,mean_speed_10,max_speed_10,min_speed_10,rms_speed_10,var_speed_10,mean_speed_5,max_speed_5,min_speed_5,rms_speed_5,var_speed_5,mean_speed_3,max_speed_3,min_speed_3,rms_speed_3,var_speed_3,mean_granulometry_10,max_granulometry_10,min_granulometry_10,rms_granulometry_10,var_granulometry_10,mean_granulometry_5,max_granulometry_5,min_granulometry_5,rms_granulometry_5,var_granulometry_5,mean_granulometry_3,max_granulometry_3,min_granulometry_3,rms_granulometry_3,var_granulometry_3,mean_solid percentage_10,max_solid percentage_10,min_solid percentage_10,rms_solid percentage_10,var_solid percentage_10,mean_solid percentage_5,max_solid percentage_5,min_solid percentage_5,rms_solid percentage_5,var_solid percentage_5,mean_solid percentage_3,max_solid percentage_3,min_solid percentage_3,rms_solid percentage_3,var_solid percentage_3,mean_HH charge cell_10,max_HH charge cell_10,min_HH charge cell_10,rms_HH charge cell_10,var_HH charge cell_10,mean_HH charge cell_5,max_HH charge cell_5,min_HH charge cell_5,rms_HH charge cell_5,var_HH charge cell_5,mean_HH charge cell_3,max_HH charge cell_3,min_HH charge cell_3,rms_HH charge cell_3,var_HH charge cell_3,mean_LL charge cell_10,max_LL charge cell_10,min_LL charge cell_10,rms_LL charge cell_10,var_LL charge cell_10,mean_LL charge cell_5,max_LL charge cell_5,min_LL charge cell_5,rms_LL charge cell_5,var_LL charge cell_5,mean_LL charge cell_3,max_LL charge cell_3,min_LL charge cell_3,rms_LL charge cell_3,var_LL charge cell_3,mean_delta HH charge cell_10,max_delta HH charge cell_10,min_delta HH charge cell_10,rms_delta HH charge cell_10,var_delta HH charge cell_10,mean_delta HH charge cell_5,max_delta HH charge cell_5,min_delta HH charge cell_5,rms_delta HH charge cell_5,var_delta HH charge cell_5,mean_delta HH charge cell_3,max_delta HH charge cell_3,min_delta HH charge cell_3,rms_delta HH charge cell_3,var_delta HH charge cell_3,mean_delta LL charge cell_10,max_delta LL charge cell_10,min_delta LL charge cell_10,rms_delta LL charge cell_10,var_delta LL charge cell_10,mean_delta LL charge cell_5,max_delta LL charge cell_5,min_delta LL charge cell_5,rms_delta LL charge cell_5,var_delta LL charge cell_5,mean_delta LL charge cell_3,max_delta LL charge cell_3,min_delta LL charge cell_3,rms_delta LL charge cell_3,var_delta LL charge cell_3,charge cell_(t-1),charge cell_(t-2),charge cell_(t-3),charge cell_(t-4),charge cell_(t-5),speed_(t-1),speed_(t-2),speed_(t-3),speed_(t-4),speed_(t-5),granulometry_(t-1),granulometry_(t-2),granulometry_(t-3),granulometry_(t-4),granulometry_(t-5),solid percentage_(t-1),solid percentage_(t-2),solid percentage_(t-3),solid percentage_(t-4),solid percentage_(t-5),HH charge cell_(t-1),HH charge cell_(t-2),HH charge cell_(t-3),HH charge cell_(t-4),HH charge cell_(t-5),LL charge cell_(t-1),LL charge cell_(t-2),LL charge cell_(t-3),LL charge cell_(t-4),LL charge cell_(t-5),delta HH charge cell_(t-1),delta HH charge cell_(t-2),delta HH charge cell_(t-3),delta HH charge cell_(t-4),delta HH charge cell_(t-5),delta LL charge cell_(t-1),delta LL charge cell_(t-2),delta LL charge cell_(t-3),delta LL charge cell_(t-4),delta LL charge cell_(t-5),cluster-edad,cluster,Edad
Timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-06-01,811.822555,3553.885205,8.877953,4400.0,52.80404,1.0,65.044377,820.0,740.0,8.177445,71.822555,812.741714,820.841222,808.370729,812.753293,18.820933,809.62674,811.822555,808.370729,809.627834,1.771907,810.234924,8

In [85]:
# Test
test=pd.concat([df_features_final["2020-04"],df_features_final["2021-09"],df_features_final["2022"]])
test.head(1)

,charge cell,TPH,speed,HH TPH,granulometry,loss of TPH,solid percentage,HH charge cell,LL charge cell,delta HH charge cell,delta LL charge cell,mean_charge cell_10,max_charge cell_10,min_charge cell_10,rms_charge cell_10,var_charge cell_10,mean_charge cell_5,max_charge cell_5,min_charge cell_5,rms_charge cell_5,var_charge cell_5,mean_charge cell_3,max_charge cell_3,min_charge cell_3,rms_charge cell_3,var_charge cell_3,mean_speed_10,max_speed_10,min_speed_10,rms_speed_10,var_speed_10,mean_speed_5,max_speed_5,min_speed_5,rms_speed_5,var_speed_5,mean_speed_3,max_speed_3,min_speed_3,rms_speed_3,var_speed_3,mean_granulometry_10,max_granulometry_10,min_granulometry_10,rms_granulometry_10,var_granulometry_10,mean_granulometry_5,max_granulometry_5,min_granulometry_5,rms_granulometry_5,var_granulometry_5,mean_granulometry_3,max_granulometry_3,min_granulometry_3,rms_granulometry_3,var_granulometry_3,mean_solid percentage_10,max_solid percentage_10,min_solid percentage_10,rms_solid percentage_10,var_solid percentage_10,mean_solid percentage_5,max_solid percentage_5,min_solid percentage_5,rms_solid percentage_5,var_solid percentage_5,mean_solid percentage_3,max_solid percentage_3,min_solid percentage_3,rms_solid percentage_3,var_solid percentage_3,mean_HH charge cell_10,max_HH charge cell_10,min_HH charge cell_10,rms_HH charge cell_10,var_HH charge cell_10,mean_HH charge cell_5,max_HH charge cell_5,min_HH charge cell_5,rms_HH charge cell_5,var_HH charge cell_5,mean_HH charge cell_3,max_HH charge cell_3,min_HH charge cell_3,rms_HH charge cell_3,var_HH charge cell_3,mean_LL charge cell_10,max_LL charge cell_10,min_LL charge cell_10,rms_LL charge cell_10,var_LL charge cell_10,mean_LL charge cell_5,max_LL charge cell_5,min_LL charge cell_5,rms_LL charge cell_5,var_LL charge cell_5,mean_LL charge cell_3,max_LL charge cell_3,min_LL charge cell_3,rms_LL charge cell_3,var_LL charge cell_3,mean_delta HH charge cell_10,max_delta HH charge cell_10,min_delta HH charge cell_10,rms_delta HH charge cell_10,var_delta HH charge cell_10,mean_delta HH charge cell_5,max_delta HH charge cell_5,min_delta HH charge cell_5,rms_delta HH charge cell_5,var_delta HH charge cell_5,mean_delta HH charge cell_3,max_delta HH charge cell_3,min_delta HH charge cell_3,rms_delta HH charge cell_3,var_delta HH charge cell_3,mean_delta LL charge cell_10,max_delta LL charge cell_10,min_delta LL charge cell_10,rms_delta LL charge cell_10,var_delta LL charge cell_10,mean_delta LL charge cell_5,max_delta LL charge cell_5,min_delta LL charge cell_5,rms_delta LL charge cell_5,var_delta LL charge cell_5,mean_delta LL charge cell_3,max_delta LL charge cell_3,min_delta LL charge cell_3,rms_delta LL charge cell_3,var_delta LL charge cell_3,charge cell_(t-1),charge cell_(t-2),charge cell_(t-3),charge cell_(t-4),charge cell_(t-5),speed_(t-1),speed_(t-2),speed_(t-3),speed_(t-4),speed_(t-5),granulometry_(t-1),granulometry_(t-2),granulometry_(t-3),granulometry_(t-4),granulometry_(t-5),solid percentage_(t-1),solid percentage_(t-2),solid percentage_(t-3),solid percentage_(t-4),solid percentage_(t-5),HH charge cell_(t-1),HH charge cell_(t-2),HH charge cell_(t-3),HH charge cell_(t-4),HH charge cell_(t-5),LL charge cell_(t-1),LL charge cell_(t-2),LL charge cell_(t-3),LL charge cell_(t-4),LL charge cell_(t-5),delta HH charge cell_(t-1),delta HH charge cell_(t-2),delta HH charge cell_(t-3),delta HH charge cell_(t-4),delta HH charge cell_(t-5),delta LL charge cell_(t-1),delta LL charge cell_(t-2),delta LL charge cell_(t-3),delta LL charge cell_(t-4),delta LL charge cell_(t-5),cluster-edad,cluster,Edad
Timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-04-01,755.74875,4188.172694,9.249551,3800.0,30.530144,0.0,65.03294,840.0,760.0,84.25125,-4.25125,770.33673,781.586053,755.74875,770.375245,59.340428,764.132392,770.226905,755.74875,764.152383,30.552526,760.475969,765.

In [86]:
tags=df_features_final.columns.to_list()
tags.remove("loss of TPH")
#tags.remove("TPH")

In [87]:
# Sobremuestreo: Aumentar numero de datos de la clase minoritaria
# ROS(Duplica clases):
ros=RandomOverSampler(random_state=0) # Random_state=0

# ROS:
Xtrain, Ytrain=ros.fit_resample(entrenamiento[tags],entrenamiento["loss of TPH"])

In [88]:
# Ytrain reg
Ytrain=Xtrain["TPH"]
Xtrain=Xtrain.drop(columns=["TPH"])

In [89]:
# Yval reg
Yval=validacion["TPH"]
Xval=validacion.drop(columns=["TPH"])

In [90]:
Ytest0=test[test["loss of TPH"]==0]["TPH"]


In [91]:
Ytest1=test[test["loss of TPH"]==1]["TPH"]


In [92]:
Xtest0=test[test["loss of TPH"]==0].drop(columns=["TPH","loss of TPH"])


In [93]:
Xtest1=test[test["loss of TPH"]==1].drop(columns=["TPH","loss of TPH"])


In [94]:
# Ytest reg
Ytest=test["TPH"]
Xtest=test.drop(columns=["TPH"])

In [95]:
#Unión del conjunto de Entrenamiento y Validación:
Entre_Vali_features=pd.concat([Xval,Xtrain])
Entre_Vali_target=pd.concat([Yval,Ytrain])

#Indices para separar ambos conjuntos:
Entre_indice=np.full(len(validacion),-1)
Vali_indice=np.full(len(entrenamiento),0)
Indices=np.append(Entre_indice,Vali_indice)
x=PredefinedSplit(Indices)

### Feature selection with XGboost & filtro alta correlación

In [63]:
# Preprocesamiento de datos

preprocessing_transformer = ColumnTransformer(
    transformers=[
        ('MinMax', MinMaxScaler(),
        Xtrain.drop(columns=["cluster","cluster-edad"]).columns.to_list()
        ),
    ])

pipe = Pipeline(
    [
        ("preprocesamiento", preprocessing_transformer), 
     ("clf", xgb.XGBRegressor(seed=1,eval_metric='rmse'
     ))
    ]
)

In [64]:
pipe.fit(Xtrain, Ytrain)

Pipeline(steps=[('preprocesamiento',
                 ColumnTransformer(transformers=[('MinMax', MinMaxScaler(),
                                                  ['charge cell', 'speed',
                                                   'HH TPH', 'granulometry',
                                                   'solid percentage',
                                                   'HH charge cell',
                                                   'LL charge cell',
                                                   'delta HH charge cell',
                                                   'delta LL charge cell',
                                                   'mean_charge cell_10',
                                                   'max_charge cell_10',
                                                   'min_charge cell_10',
                                                   'rms_charge cell_10',
                                                   'var_charge cell_10',
          

In [65]:
Ypred= pipe.predict(Xtest)

# Métricas
print('R2: ',r2_score(Ytest, Ypred),'\n','RMSE: ',mean_squared_error(Ytest, Ypred, squared=False))

R2:  0.5686737491432412 
 RMSE:  277.6700995666607


In [67]:
df_importance=pd.DataFrame({"tag":Xtrain.drop(columns=["cluster","cluster-edad"]).columns,"importance":pipe["clf"].feature_importances_}).sort_values(by="importance",ascending=False).reset_index(drop=True)
df_importance.head()

,tag,importance
0,HH TPH,0.074089
1,max_delta LL charge cell_3,0.068203
2,min_solid percentage_5,0.066690
3,min_solid percentage_10,0.054930
4,min_speed_5,0.051497


In [68]:
pd.options.plotting.backend = "plotly"
fig=df_importance["importance"].plot(kind='hist')
fig.update_layout(height=500, width=1200,title="Feature importance XGBoost (Gain)",) 
fig.show()

In [69]:
# Plot
fig = go.Figure([go.Bar(x=df_importance["tag"], y=df_importance["importance"])])
fig.update_layout(height=500, width=1200,title="Feature importance XGBoost (Gain)",) 
fig.show()

In [70]:
Xtrain.shape

(976434, 171)

In [71]:
xgboost_fs=df_importance[:100].tag.to_list()

In [72]:
# Función que filtra tags por alta correlación
def corrkill(dataframe, features, corr_cut=0.8):

    df = dataframe[features]
    dfcorr = pd.DataFrame(np.triu(df.corr()), columns=df.columns, index=df.columns)
    dfcorr = dfcorr.stack().reset_index()
    dfcorr.columns = ['Feat1','Feat2','Val']
    dfcorr = dfcorr[~dfcorr['Val'].isin([0,1])]
    dfcorr = dfcorr[dfcorr['Val'].abs()>corr_cut]
    dfcorr["Val"]=dfcorr["Val"].abs()
    
    return list(dfcorr['Feat1']), dfcorr.sort_values(by="Val",ascending=False)

In [73]:
# Se obtienen tags altamente correlacionados y matriz que permite visualizar cuales son los par de tags altamente correlacionados
collinearity, table_cor=corrkill(Xtrain.drop(columns=["cluster","cluster-edad"]),xgboost_fs,corr_cut=0.9)

In [74]:
tag_select = [i for i in xgboost_fs if i not in collinearity]
tag_select

['HH TPH',
 'var_speed_10',
 'var_solid percentage_3',
 'var_solid percentage_5',
 'var_speed_5',
 'var_charge cell_3',
 'var_solid percentage_10',
 'speed_(t-5)',
 'charge cell_(t-3)',
 'mean_HH charge cell_5',
 'var_delta HH charge cell_10',
 'var_delta HH charge cell_3',
 'granulometry_(t-2)',
 'var_granulometry_3',
 'solid percentage_(t-5)',
 'rms_delta HH charge cell_5',
 'rms_delta LL charge cell_5']

In [75]:
# Preprocesamiento de datos

preprocessing_transformer = ColumnTransformer(
    transformers=[
        ('MinMax', MinMaxScaler(),
        tag_select
        ),
    ])

pipe = Pipeline(
    [
        ("preprocesamiento", preprocessing_transformer), 
     ("clf", xgb.XGBRegressor(seed=1,eval_metric='rmse'
     ))
    ]
)

In [76]:
pipe.fit(Xtrain, Ytrain)

Pipeline(steps=[('preprocesamiento',
                 ColumnTransformer(transformers=[('MinMax', MinMaxScaler(),
                                                  ['HH TPH', 'var_speed_10',
                                                   'var_solid percentage_3',
                                                   'var_solid percentage_5',
                                                   'var_speed_5',
                                                   'var_charge cell_3',
                                                   'var_solid percentage_10',
                                                   'speed_(t-5)',
                                                   'charge cell_(t-3)',
                                                   'mean_HH charge cell_5',
                                                   'var_delta HH charge '
                                                   'cell_10',
                                                   'var_delta HH charge cell_3',
         

In [77]:
Ypred= pipe.predict(Xtest)

# Métricas
print('R2: ',r2_score(Ytest, Ypred),'\n','RMSE: ',mean_squared_error(Ytest, Ypred, squared=False))

R2:  0.5235632179851912 
 RMSE:  291.82924535524944


### Prueba de modelo con variables categoricas

In [103]:
# Preprocesamiento de datos

preprocessing_transformer = ColumnTransformer(
    transformers=[('OneHotEncoder', OneHotEncoder(handle_unknown="ignore",sparse=False),["cluster"]),
        ('MinMax', MinMaxScaler(),tag_select+["Edad"])
    ])

pipe = Pipeline(
    [
        ("preprocesamiento", preprocessing_transformer), 
     ("clf", xgb.XGBRegressor(seed=1,eval_metric='rmse'
     ))
    ]
)

In [100]:
pipe.fit(Xtrain, Ytrain)

Pipeline(steps=[('preprocesamiento',
                 ColumnTransformer(transformers=[('OneHotEncoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  ['cluster']),
                                                 ('MinMax', MinMaxScaler(),
                                                  ['HH TPH', 'var_speed_10',
                                                   'var_solid percentage_3',
                                                   'var_solid percentage_5',
                                                   'var_speed_5',
                                                   'var_charge cell_3',
                                                   'var_solid percentage_10',
                                                   'speed_(t-5)',
                                                   'charge cell_(t-3)',
   

**General**

In [102]:
Ypred= pipe.predict(Xtest)

# Métricas
print('R2: ',r2_score(Ytest, Ypred),'\n','RMSE: ',mean_squared_error(Ytest, Ypred, squared=False))

R2:  0.554439629123675 
 RMSE:  282.2145814870526


# Grid search

In [105]:
from sklearn.feature_selection import SelectPercentile, f_regression,mutual_info_regression

In [109]:
# Preprocesamiento de datos

preprocessing_transformer = ColumnTransformer(
    transformers=[
        ('MinMax', MinMaxScaler(),
        Xtrain.drop(columns=["cluster","cluster-edad"]).columns
        #features.columns.to_list()
        ),
    ])

pipe = Pipeline(
    [
        ("preprocesamiento", preprocessing_transformer), 
        ("Selection", SelectPercentile(f_regression, percentile=10)),
        #("PCA",PCA(n_components=150 )), #0.08
     ("reg", xgb.XGBRegressor(seed=1,eval_metric='rmse'))
    ]
)


param_grid_reg = {#'reg__learning_rate': list(np.linspace(0.001,1,10)), # Boosting learning rate
                  #'reg__gamma':list(np.linspace(0.0001,50,5)), # Minimum loss reduction required to make a further partition on a leaf node of the tree.
                  #'reg__max_depth':[5,10,12,15,20], # Maximum tree depth for base learners.
                  #'reg__min_child_weight':[10,5,1,2,3], # Minimum sum of instance weight(hessian) needed in a child.
                  #'reg__colsample_bytree':[0.5,0.8,1], # Subsample ratio of columns when constructing each tree.
                  #'reg__n_estimators': [100,200,300], # Number of gradient boosted trees.   
                  #'reg__reg_alpha': [0,0.1,0.2,0.4,0.8,1.6,3.2,6.4,12.8,25.6,51.2,102.4,200],
              #'reg__reg_lambda': [0,0.1,0.2,0.4,0.8,1.6,3.2,6.4,12.8,25.6,51.2,102.4,200],
               'Selection__percentile': [10,20,30,50,70,80,100]              
                 } 

# Clasificación
gs_reg = HalvingGridSearchCV(pipe, param_grid_reg, scoring='neg_mean_absolute_error',cv=x,refit=True,
                             verbose=10,
                             aggressive_elimination=True
                            )

# Fit
gs_reg.fit(Entre_Vali_features, Entre_Vali_target)


n_iterations: 2
n_required_iterations: 2
n_possible_iterations: 2
min_resources_: 349805
max_resources_: 1049417
aggressive_elimination: True
factor: 3
----------
iter: 0
n_candidates: 7
n_resources: 349805
Fitting 1 folds for each of 7 candidates, totalling 7 fits
[CV 1/1; 1/7] START Selection__percentile=10....................................
[CV 1/1; 1/7] END Selection__percentile=10;, score=(train=-113.007, test=-194.088) total time=   3.6s
[CV 1/1; 2/7] START Selection__percentile=20....................................
[CV 1/1; 2/7] END Selection__percentile=20;, score=(train=-86.097, test=-252.556) total time=   5.3s
[CV 1/1; 3/7] START Selection__percentile=30....................................
[CV 1/1; 3/7] END Selection__percentile=30;, score=(train=-67.807, test=-402.917) total time=   8.6s
[CV 1/1; 4/7] START Selection__percentile=50....................................
[CV 1/1; 4/7] END Selection__percentile=50;, score=(train=-67.871, test=-403.678) total time=  10.0s
[CV 1

HalvingGridSearchCV(aggressive_elimination=True,
                    cv=PredefinedSplit(test_fold=array([-1, -1, ...,  0,  0])),
                    estimator=Pipeline(steps=[('preprocesamiento',
                                               ColumnTransformer(transformers=[('MinMax',
                                                                                MinMaxScaler(),
                                                                                Index(['charge cell', 'speed', 'HH TPH', 'granulometry', 'solid percentage',
       'HH charge cell', 'LL charge cell', 'delta HH charge cell',
       'delta LL charge cell', 'mean_c...
                                                            monotone_constraints=None,
                                                            n_estimators=100,
                                                            n_jobs=None,
                                                            num_parallel_tree=None,
                              

In [110]:
Ypred= gs_reg.predict(Xtest)

# Métricas
print('R2: ',r2_score(Ytest, Ypred),'\n','RMSE: ',mean_squared_error(Ytest, Ypred, squared=False))

R2:  0.479722714438238 
 RMSE:  304.96050887261714
